In [1]:
import numpy as np
import pandas as pd
import networkx as nx

from sklearn.cluster import KMeans
from sklearn import manifold

from scipy.optimize import minimize
from scipy.spatial.distance import pdist, squareform

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
orderdf = pd.read_csv('ignored_assets/orders.csv', parse_dates = [6,7])


In [3]:
coredf = orderdf[orderdf['dest_district_hash'].isin(orderdf['start_district_hash'].unique())].copy()


In [4]:
# Focus on a given date
date_range=pd.Period('2016-01-21','D')
coredf['dayts']=coredf['timestamp'].dt.to_period('D')


In [5]:
# For some reason this, of all things, takes forever. Input on how to make this faster is very welcome
myorders = coredf[(coredf['dayts']==date_range) & (~pd.isnull(coredf['driver_id']))].copy()

In [6]:
myorders=myorders.sort_values(by='timestamp')

In [7]:
posdf=pd.read_csv('inferred_locations.csv')
posdf=posdf.set_index('start_district_hash')

In [8]:
myorders['month']=myorders['timestamp'].dt.month
myorders['day']=myorders['timestamp'].dt.day
myorders['hour']=myorders['timestamp'].dt.hour
myorders['minute']=myorders['timestamp'].dt.minute
myorders['second']=myorders['timestamp'].dt.second

In [9]:
myorders['start_x']=posdf.loc[myorders['start_district_hash'][:]].x.values
myorders['start_y']=posdf.loc[myorders['start_district_hash'][:]].y.values
myorders['dest_x']=posdf.loc[myorders['dest_district_hash'][:]].x.values
myorders['dest_y']=posdf.loc[myorders['dest_district_hash'][:]].y.values

In [10]:
MATLABorders=myorders[['month', 'day','hour','minute','second','start_x','start_y','dest_x','dest_y','expected_travel_time']]

In [11]:
MATLABorders.to_csv('ignored_assets/MATLAB_orders.csv')

In [12]:
MATLABorders

,month,day,hour,minute,second,start_x,start_y,dest_x,dest_y,expected_travel_time
8373229,1,21,0,0,5,-6.762865,-2.550955,-2.067771,-7.821290,14.463949
8087724,1,21,0,0,6,2.084387,-4.174350,-2.067771,-7.821290,2.799474
8486145,1,21,0,0,6,2.123738,-11.838793,9.373054,-17.069085,6.065527
8354937,1,21,0,0,7,-29.190246,3.030521,-29.190246,3.030521,5.598948
8405053,1,21,0,0,8,18.725480,7.920077,18.725480,7.920077,5.132369
8513991,1,21,0,0,8,-4.670943,22.803132,12.111844,24.608489,23.795529
8350975,1,21,0,0,10,2.123738,-11.838793,2.123738,-11.838793,6.065527
8495444,1,21,0,0,11,2.123738,-11.838793,2.123738,-11.838793,8.398422
8118574,1,21,0,0,13,-2.352430,2.706548,-1.627176,13.560078,7.931843
8236500,1,21,0,0,14,2.084387,-4.174350,2.084387,-4.174350,2.332895
